In [1]:
import json

with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BitsAndBytesConfig

import torch
import gc

torch.cuda.empty_cache()
gc.collect()

## `Google-Flan-t5-xl`
    # T5 Modeli
        ## Text-To-Text Transfer Transformer: NLP görevlerini `text-to-text` formatında ele alır
        ## Esneklik: Modeli çeşitli görevlerde (metin sınıflandırma, çeviri, özetleme, soru-cevap) esnek ve güçlü hale getirir.
        ## Transfer Learning: Geniş bir veri kümesi üzerinde önceden eğitilir ve daha sonra belirli görevler için ince ayar yapılır.

    # Flan: Modelin performansını artırmak için ek ince ayar ve eğitim adımlarını içerir.
        ## Boyut: XL model, daha fazla parametre içerir ve bu da onun daha büyük ve daha güçlü bir model olmasını sağlar.
        ## Gelişmiş Performans: Daha büyük boyutu ve ince ayar süreçleri sayesinde, Flan-T5 XL, dil anlama, metin üretimi ve diğer NLP görevlerinde oldukça yüksek performans gösterir.

double_quant_config  = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, llm_int8_enable_fp32_cpu_offload=True)

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto", quantization_config= double_quant_config)

def build_prompt(query, search_results):
    context = ""

    prompt_template = \
    """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from th CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: {context}
    """.strip()

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context)

    return prompt


def build_elasticsearch(index_settings, index_name, documents):
    es_client = Elasticsearch('http://127.0.0.1:9200')

    es_client.indices.delete(index=index_name, ignore_unavailable=True)
    es_client.indices.create(index=index_name, body=index_settings)

    for doc in tqdm(documents):
        es_client.index(index=index_name, document=doc)

    return es_client


def es_search(es_client, search_query, index_name):
    result_docs = []
    response = es_client.search(index=index_name, body=search_query)

    for hit in response['hits']['hits']:
       result_docs.append(hit['_source'])

    return result_docs


def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

## max_length = Daha uzun yanıtlar için, daha yüksek değerler ayarlayın.
    ## num_beams = Bu değeri artırmak, olası dizilerin daha ayrıntılı olarak araştırılmasına yol açar. Tipik değerler 5 ile 10 arasındadır.
    ## do_sample = Örnekleme yöntemlerini kullanmak için bunu True ayarlayın. Bu, daha çeşitli yanıtlar üretebilir.
    ## temperature = Bu değeri düşürmek, modeli daha emin ve deterministik yapar, daha yüksek değerleri ise çeşitliliği artırır. Tipik değerler 0.7 ile 1.5 arasındadır.
    ## top_k ve top_p = Bu parametreler çekirdek örneklemeyi kontrol eder. `top_k` örnekleme havuzunu en üst `k` token ile sınırlar, `top_p` ise kümülatif olasılığı kullanarak örnekleme havuzunu keser. Bu parametreleri istenen rastgelelik seviyesine göre ayarlayın.

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", True),
        temperature=generate_params.get("temperature", 0.5),
        top_k=generate_params.get("top_k", 1),
        top_p=generate_params.get("top_p", 0.95),)
    
    result = tokenizer.decode(outputs[0])

    return result


def rag(prompt):
    answer = llm(prompt)

    return answer

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }, 
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "elasticsearch-question"

query = "I just discover the course. Can I still join it ?"

search_query = {
  "size": 5,
  "query": {
    "bool": {
      "must": {
        "multi_match": {
          "query": query,
          "fields": ["question^3", "text", "section"],
          "type": "best_fields"
        }
      },
      "filter": {
        "term": {
          "course": "data-engineering-zoomcamp"
        }
      }
    }
  }
}

In [4]:
es_client = build_elasticsearch(index_settings, index_name, documents)

  0%|          | 0/948 [00:00<?, ?it/s]

In [15]:
results = es_search(es_client, search_query, index_name)
prompt = build_prompt(query, results)

In [16]:
answer = rag(prompt)

In [18]:
answer

"<pad> Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.</s>"